# <center>Workflow for the CRC1333 project B07 - Technical Chemistry</center>
# <center>2.2 Experimental notebook - Analysis</center>

---

This is the ``Experimental`` ``notebook``, where the actual analysis of the experiments takes place. It consists of three parts: ``Parsing``, ``analysis`` and ``DaRUS`` ``upload``. Within the scope of each project, multiple experiments are perfomed, hence multiple analyses are to be done. For each individual experiment this workflow is to be executed once, and the results can be appended to the project's dataset.

---

In [1]:
# from sdRDM.generator import generate_python_api
from sdRDM import DataModel

In [2]:
# generate_python_api('specifications/datamodel_b07_tc.md', '', 'datamodel_b07_tc')

Import standard library python packages.

In [3]:
%reload_ext autoreload
%autoreload 2

from datamodel_b07_tc.tools import FaradayEfficiencyCalculator
from datamodel_b07_tc.tools import PeakAreaAssignment
# from DEXPI2sdRDM import DEXPI2sdRDM

In [4]:
import os
import json
import ipywidgets as widgets
import logging
import logging.config
from IPython.display import display
from pathlib import Path

---
## Section 0: Paths and Logging
---

Get path to the directory this notebook is located and check for correctness.

In [5]:
root = Path(os.path.abspath(''))
print("Path to this notebook's location:", root)
print('Is the path valid?', root.is_dir())

Path to this notebook's location: /mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc
Is the path valid? True


Set path for the logger.

In [6]:
config_path = root / "datamodel_b07_tc/tools/logging/config.json"
print(config_path)

/mnt/c/Users/rscho/Documents/GitHub/datamodel_b07_tc/datamodel_b07_tc/tools/logging/config.json


Set up logger.

In [7]:
logging_config_path = root / "datamodel_b07_tc/tools/logging/config.json"
with open(logging_config_path) as logging_config_json:
    logging_config = json.load(logging_config_json)

In [8]:
logging.config.dictConfig(logging_config)

In [9]:
logger = logging.getLogger(__name__)
logger.debug("obacht")
logger.warning('uff')

2023-09-19 15:20:02,956 - __main__ - WARNING - uff


---
## Section 1: Parsing dataset
---

In this section the data model and the dataset as well as all the output files necessary for analysis are parsed.  

 Set path to datasets.

In [10]:
path_to_datasets = root / 'datasets'

List all available datasets in the directory.


In [11]:
files = path_to_datasets.iterdir()
json_files = {index:file for index, file in enumerate(files) if file.suffix == '.json'}
for index, file in json_files.items():
    print(f'{index}: {file.name}')

0: b07.json


Choose dataset to be loaded by its index.

In [12]:
index_dataset = 0
dataset, lib = DataModel.parse(json_files[index_dataset])

Visualize the data model.

In [13]:
# lib.Dataset.meta_tree()

Print current status of the dataset.

In [14]:
# print(dataset.json())

---
## Section 2: Analysis
---

In [15]:
index_experiment = 0

In [16]:
experiment = dataset.experiments[index_experiment]

In [17]:
print(experiment)

Experiment
├── id = experiment0
├── plant_setup
│   └── PlantSetup
│       └── id = plantsetup0
├── measurements
│   ├── 0
│   │   └── Measurement
│   │       ├── id = measurement1
│   │       ├── measurement_type = Potentiostatic measurement
│   │       └── metadata
│   │           ├── 0
│   │           │   └── Metadata
│   │           │       ├── id = metadata1
│   │           │       ├── parameter = PSTAT
│   │           │       ├── value = REF3000-19129
│   │           │       ├── data_type = PSTAT
│   │           │       └── description = Potentiostat
│   │           ├── 1
│   │           │   └── Metadata
│   │           │       ├── id = metadata2
│   │           │       ├── parameter = IINIT
│   │           │       ├── value = -2.00000E+002
│   │           │       ├── data_type = QUANT
│   │           │       └── description = Initial I (mA/cm^2)
│   │           ├── 2
│   │           │   └── Metadata
│   │           │       ├── id = metadata3
│   │           │       ├── parameter

Assign peak areas to species.

The peak areas recorded by the GC have to be matched with the correct species. The individial ``Area`` is selected by its corresponding ``Peak_Number``. It is possible that the same species is accountable for multiple peaks, i.d. multiple peaks are assigned to the same species.


### Peak assignment

Get list of alll three GC measurements.

In [18]:
gc_measurements = experiment.get("measurements", "measurement_type", "GC measurement")[0]

Create list for all the three dictionaries of the assigned peak areas. 

In [19]:
list_of_assigned_peak_areas = []

First GC Measurement.

In [20]:
first_gc_measurement = gc_measurements[0]

In [21]:
peak_area_assignment = PeakAreaAssignment.from_gc_measurement(gc_measurement=first_gc_measurement)
peak_areas_index_dict = peak_area_assignment.get_peak_areas_index_dict
for index, peak_area in peak_areas_index_dict.items():
    print(f'{index}:{peak_area}')

1:69.1715774536133
2:65492.74609375
3:164.157028198242
4:141.173934936524
5:1624.07373046875
6:2876.95263671875
7:43.7316970825196


In [22]:
peak_assignment_dict={
    'Hydrogen': [1],
    'Carbon dioxide': [2],
    'Carbon monoxide': [6],
    'Methane': [3],
    # 'C2H4': [5],
    # 'C2H6': [4],
}

In [23]:
assigned_peak_areas_dict = peak_area_assignment.assign(peak_assignment_dict=peak_assignment_dict)
list_of_assigned_peak_areas.append(assigned_peak_areas_dict)
for species, peak_area in assigned_peak_areas_dict.items():
    print(f'{species}:{peak_area}')

Hydrogen:69.1715774536133
Carbon dioxide:65492.74609375
Carbon monoxide:2876.95263671875
Methane:164.157028198242


Second GC Measurement.

In [24]:
second_gc_measurement = gc_measurements[1]

In [25]:
peak_area_assignment = PeakAreaAssignment.from_gc_measurement(gc_measurement=second_gc_measurement)
peak_areas_index_dict = peak_area_assignment.get_peak_areas_index_dict
for index, peak_area in peak_areas_index_dict.items():
    print(f'{index}:{peak_area}')

1:104.630867004395
2:70813.515625
3:317.436950683594
4:160.268005371094
5:1518.81433105469
6:3685.70336914063
7:45.592227935791


In [26]:
peak_assignment_dict={
    'Hydrogen': [1],
    'Carbon dioxide': [2],
    'Carbon monoxide': [6],
    'Methane': [3],
    # 'C2H4': [5],
    # 'C2H6': [4],
}

In [27]:
assigned_peak_areas_dict = peak_area_assignment.assign(peak_assignment_dict=peak_assignment_dict)
list_of_assigned_peak_areas.append(assigned_peak_areas_dict)
for species, peak_area in assigned_peak_areas_dict.items():
    print(f'{species}:{peak_area}')

Hydrogen:104.630867004395
Carbon dioxide:70813.515625
Carbon monoxide:3685.70336914063
Methane:317.436950683594


Third GC Measurement.

In [28]:
third_gc_measurement = gc_measurements[2]

In [29]:
peak_area_assignment = PeakAreaAssignment.from_gc_measurement(gc_measurement=third_gc_measurement)
peak_areas_index_dict = peak_area_assignment.get_peak_areas_index_dict
for index, peak_area in peak_areas_index_dict.items():
    print(f'{index}:{peak_area}')

1:97.2599182128906
2:71603.8515625
3:317.305053710938
4:159.882797241211
5:1266.77111816406
6:3433.341796875
7:39.0086631774903


In [30]:
peak_assignment_dict={
    'Hydrogen': [1],
    'Carbon dioxide': [2],
    'Carbon monoxide': [6],
    'Methane': [3],
    # 'C2H4': [5],
    # 'C2H6': [4],
}

In [31]:
assigned_peak_areas_dict = peak_area_assignment.assign(peak_assignment_dict=peak_assignment_dict)
list_of_assigned_peak_areas.append(assigned_peak_areas_dict)
for species, peak_area in assigned_peak_areas_dict.items():
    print(f'{species}:{peak_area}')

Hydrogen:97.2599182128906
Carbon dioxide:71603.8515625
Carbon monoxide:3433.341796875
Methane:317.305053710938


Print current state of the experiment object.

In [32]:
# print(experiment.json())

### Calculation

Set up the ``FaradayEfficiencyCalculator``.

In [33]:
calculator = FaradayEfficiencyCalculator(
    experiment=experiment,
    electrode_surface_area=1.0
)

Set averaging radius.

In [34]:
mean_radius = 10

Calculate faraday efficiencies.

In [35]:
faraday_efficiencies = []
for gc_measurement, assigned_peak_areas_dict in zip(gc_measurements, list_of_assigned_peak_areas):
    faraday_efficiency = calculator.calculate_faraday_efficiency(
        gc_measurement=gc_measurement,
        mean_radius=mean_radius,
        assigned_peak_areas_dict=assigned_peak_areas_dict
    )
    faraday_efficiencies.append(faraday_efficiency)


2023-02-06 09:58:48
2023-02-06 09:58:48
2023-02-06 09:58:48


In [36]:
import pandas as pd
mean_faraday_efficiency = pd.concat(faraday_efficiencies).groupby(level=0).mean()
mean_faraday_efficiency

,Faraday_efficiency
Carbon dioxide,252.970554
Carbon monoxide,6.524397
Ethane,NaN
Ethene,NaN
Hydrogen,17.5246
Methane,7.229418


In [37]:
# volumetric_flow_mean_list = []
# for gc_measurement in gc_measurements:
#     volumetric_flow_mean=calculator.get_volumetric_flow_mean(gc_measurement=gc_measurement)
#     volumetric_flow_mean_list.append(volumetric_flow_mean)
# for assigned_peak_areas_dict in list_of_assigned_peak_areas:
#     volumetric_flow_mean = calculator.calculate(assigned_peak_areas_dict=assigned_peak_areas_dict, correction_factors_dict=correction_factors_dict)
#     volumetric_flow_mean_list.append(volumetric_flow_mean)
#     print(volumetric_flow_mean)
# initial_time = calculator.get_initial_time()
# initial_current = calculator.get_initial_current()

    # calculator.calculate_faraday_efficiency(gc_measurement)

Calculate the ``conversion`` ``factor`` using the correction factors.

Get ``volumetric`` ``flow`` ``mean`` in ml/min at the time of the GC measurement.

The mass flow at the time of the GC measurement is determined by matching the time of the gc measurement with the corresponding times of the mass flow measurements. Errors in the mass flows due to strong fluctuations are minimized by calculating the mean by averaging over a certain number (=``radius``) of measuring points before and after the time of the GC measurement. The radius has to be specified in accordance with the strength of fluctuations.

Calculate volumetric flow fractions in %.

Calculate material flow in mmol/min.

Get initial current in mA and initial time in s.

Calculate theoretical material flow in mmol/min.

Calculate Faraday efficiency and load into dataset.

---
## DaRUS upload
---

In [39]:
dataset.experiments.append(experiment)

In [40]:
with open(json_files[index_dataset], "w") as f:
    f.write(dataset.json())

In [41]:
button = widgets.Button(description="Append experiment", layout=widgets.Layout(width='30%', height='80px'))
button.style.button_color = 'darkcyan'
button.style.text_color = 'lightgrey'
button.style.font_size = '30px'


output = widgets.Output()

display(button, output)

def click_on_button(b):
    with output:
        print("Experiment successfully appended.")

button.on_click(click_on_button)

Button(description='Append experiment', layout=Layout(height='80px', width='30%'), style=ButtonStyle(button_co…

Output()

In [42]:
# %%html
# <style>
# .cell-output-ipywidget-background {
#    background-color: transparent !important;
# }
# .cell-output-ipywidget-foreground {


    
# .jp-OutputArea-output {
#    background-color: transparent;
# }  
# </style>

In [43]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [44]:
# print(f'type of "volumetric_fractions": {type(volumetric_fractions)}')
# print(f'type of "conversion_factor": {type(conversion_factor)}')
# print(f'type of "real_volumetric_flow": {type(real_volumetric_flow)}')
# print(f'type of "volumetric_flow_fractions": {type(volumetric_flow_fractions)}')
# print(f'type of "real_volumetric_flow": {type(real_volumetric_flow)}')
# print(f'type of "theoretical_material_flow": {type(theoretical_material_flow)}')
# print(f'type of "material_flow": {type(material_flow)}')

In [45]:
# volumetric_fractions = calculator.volumetric_fractions
# volumetric_fractions
# conversion_factor = calculator.conversion_factor
# conversion_factor
# real_volumetric_flow = calculator.real_volumetric_flow
# real_volumetric_flow
# volumetric_flow_fractions = calculator.volumetric_flow_fractions
# volumetric_flow_fractions
# material_flow = calculator.material_flow
# material_flow
# theoretical_material_flow = calculator.theoretical_material_flow
# theoretical_material_flow